In [1]:
# -------------------------- LIBRARIES NECESSARY IN THIS PROJECT  -------------------------- #
import pandas as pd
import numpy as np

# -------------------------- FUNCTIONS -------------------------- #

def getDataSet(data_path):
    return pd.read_csv(data_path)

# -------------------------- PATH OF THE DATASETS USED IN THE PROJECT  -------------------------- #

invoices_claims_last_actived_path="dataSets/Invoices_Claims_Last_Actived.csv"
invoices_claims_last_completed_path="dataSets/Invoices_Claims_Last_Completed.csv"
invoices_claims_path="dataSets/Invoices_Claims.csv"
members_supported="dataSets/Members_Supported.csv"
plans_budgets="dataSets/Plans_Budgets.csv"
providers="dataSets/Providers.csv"

## 1. Data manipulation and cleansing<a name="preparation"></a>

### Upload dataSets

In [2]:
# upload the dataSet with invoices and claims data
invoices_claims_last_actived_all_fields = getDataSet(invoices_claims_last_actived_path)
invoices_claims_last_completed_all_fields = getDataSet(invoices_claims_last_completed_path)

# upload the dataSet with invoices and claims data
df_plans_budgets_all_fields = getDataSet(plans_budgets)

# upload the dataSet with invoices and claims data
df_members_supported_all_fields = getDataSet(members_supported)

C:\Users\dttt1\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (37,41,42,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\dttt1\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


### Members dataset cleasing

In [3]:
# Select just the fields to be used
df_members_supported = df_members_supported_all_fields[{"id"
                                                      , "member_key" 
                                                      , "first_name"
                                                      , "last_name"
                                                      , "price_zone_code"
                                                      , "u_disabilities"
                                                      , "SA1"
                                                      }]

# Check if there any Nan Field
df_any_Nan = df_members_supported.isna().any()

#  There are some u_disabilities, SA1 NaN
# However, all records are keeped
df_any_Nan

# And it is assigned 'No Assigned' to both u_disabilities and SA1
df_members_supported.loc[df_members_supported['u_disabilities'].isnull(),['u_disabilities']] = "No Assigned"
df_members_supported.loc[df_members_supported['SA1'].isnull(),['SA1']] = "No Assigned"

# Rename some dataSet colums to create a name's pattern
df_members_supported.rename(columns={'id':'member_id'
                                   , 'u_disabilities':'disabilities'
                                    } , inplace = True)

df_members_cleased = df_members_supported

C:\Users\dttt1\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\dttt1\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### Plains and budgets dataset cleasing

In [4]:
# Select just the fields to be used
df_plans_budgets = df_plans_budgets_all_fields[{"member_key"
                                              , "plan_key"
                                              , "plan_start_date" 
                                              , "plan_start_date.1"
                                              , "status"
                                              , "budget_number"
                                              , "level2_key"
                                              , "level2_name"
                                              , "level1_key"
                                              , "level1_name"
                                              , "item_category_level3_key"
                                              , "name_stated_item"
                                              , "value_allocated_budget"
                                              , "status_budget"
                                              }]


# Rename some dataSet colums to create a name's pattern
df_plans_budgets.rename(columns={'plan_start_date.1':'plan_end_date'
                               , 'status':'plan_status'                               
                               , 'status_budget':'budget_status'
                               , 'value_allocated_budget':'budget_value_allocated'
                               , 'level2_key':'budget_level2_key'
                               , 'level2_name':'budget_level2_name'
                               , 'level2_display_name':'budget_level2_display_name'
                               , 'level1_key':'budget_level1_key'
                               , 'level1_name':'budget_level1_name'
                               , 'item_category_level3_key':'budget_level3_key'
                               , 'level3_reference_number':'budget_level3_reference_number'
                               , 'name_stated_item':'budget_level3_name'                               
                                } , inplace = True)


# It keeps all plain, but deletes the no CONFIRMED budgets
df_plans_budgets.drop(df_plans_budgets[df_plans_budgets.budget_status != 'CONFIRMED'].index)

# Check if there is any NaN field
df_any_Nan = df_plans_budgets.isna().any()
df_any_Nan

# if level3_key is null
#   Assigns 'no assigned' in columns 'level3_key','level3_reference_number', 'stated_item_name'
#    Because some budgts starts and level2 and, so level3 is null
df_plans_budgets.loc[df_plans_budgets['budget_level3_key'].isnull(),['budget_level3_key', 'budget_level3_name']] = "No Assigned"

# Check if there is any NaN field
df_any_Nan = df_plans_budgets.isna().any()
df_any_Nan

# Check the why some budget_number are NaN
 # And figured out that they are null 
 # because they are records created manually by users for balance
budget_number_NaN = df_plans_budgets[df_plans_budgets["budget_number"].isna()]
budget_number_NaN

# Deletes the records with budget_number are NaN 
#  And keeps the ones with level3_reference_number, stated_item_name, level3_key
#   because if level3_reference_number, stated_item_name, level3_key are NaN
#   Means the budget starts in level 2
df_plans_budgets = df_plans_budgets.drop(df_plans_budgets[df_plans_budgets.budget_number.isna()].index)

df_plans_budgets['plan_start_date'] = df_plans_budgets['plan_start_date'].astype('datetime64[ns]').dt.date
df_plans_budgets['plan_end_date'] = df_plans_budgets['plan_end_date'].astype('datetime64[ns]').dt.date


# Group the plans from their bugdets
df_plans_budgets_cleased = df_plans_budgets.groupby(["member_key"
                                                   , "plan_key"
                                                   , "plan_status"
                                                   , "plan_start_date"
                                                   , "plan_end_date"
                                                   , "budget_status"
                                                   , "budget_level3_name"
                                                   , "budget_level3_key" 
                                                   , "budget_level2_key"
                                                   , "budget_level2_name"
                                                   , "budget_level1_key"
                                                   , "budget_level1_name"]).agg({"budget_value_allocated": "sum"}).reset_index()                             

### Invoice and claims dataset cleasing

In [5]:
# Select just the fields to be used
invoices_claims_last_actived = invoices_claims_last_actived_all_fields[{"member_id"
                                                                      , "invoice_state"
                                                                      , "claim_state"
                                                                      , "key"  
                                                                      , "key.1"
                                                                      , "key.2"
                                                                      , "claim_start_date"
                                                                      , "claim_end_date"
                                                                      , "claim_funded_amount"  
                                                                      , "funded_total"
                                                                      , "invoiced_units"
                                                                      , "invoiced_unit_price"
                                                                      }]

# Rename some dataSet colums to create a name's pattern
invoices_claims_last_actived.rename(columns={'funded_total':'invoiced_amount'
                                           , 'claim_funded_amount':'funded_amount'
                                           , 'key.1':'claim_level2_key'
                                           , 'key.2':'claim_level1_key'
                                           , 'key':'claim_level3_key'
                                              } , inplace = True)


invoices_claims_last_actived['invoiced_amount'] = invoices_claims_last_actived['invoiced_unit_price'] * invoices_claims_last_actived['invoiced_units'] 


# Get the name of columns with Nan values
#  and figured out that some fields are Nan values
df_any_Nan = invoices_claims_last_actived.isna().any()
df_any_Nan

# There are 'funded_amount' filds == NaN
#   Checking their types it is seing that they are float64 typed 
#invoices_claims_last_actived.info()

# Check The NaN fields
df_invoices_claims_filtered = invoices_claims_last_actived[invoices_claims_last_actived.isna().any(axis=1)]

# Check state of claims with NaN claim_state 
df_filtered = df_invoices_claims_filtered[{"invoice_state"
                                         , "claim_state"
                                         }]

# Group the state of invoice and claim of all null data
#  And Keeps these records
#   Because if 'funded_amount' is NaN means the claims is not PAID OR REFUNDED
df_filtered_grouped = df_invoices_claims_filtered.groupby(["invoice_state"
                                                         , "claim_state"
                                                          ]).count()

# Some funded_amount == null, then they will be assinged as 0 and will not be deleted
# Because if they are null means they are still waiting for approval
invoices_claims_last_actived.loc[invoices_claims_last_actived['funded_amount'].isnull(),['funded_amount']] = 0

#invoices_claims_last_actived['invoice_created_at'] = invoices_claims_last_actived['invoice_created_at'].astype('datetime64[ns]').dt.date
invoices_claims_last_actived['claim_start_date'] = invoices_claims_last_actived['claim_start_date'].astype('datetime64[ns]').dt.date
invoices_claims_last_actived['claim_end_date'] = invoices_claims_last_actived['claim_end_date'].astype('datetime64[ns]').dt.date

# Group the invoices from their claims
invoices_claims_last_actived_cleased = invoices_claims_last_actived.groupby(["member_id"
                                                                           , "invoice_state" 
                                                                           , "claim_state"
                                                                           , "claim_level1_key"
                                                                           , "claim_level2_key"
                                                                           , "claim_level3_key"]).agg({"invoiced_amount": "sum", "funded_amount": "sum"}).reset_index()                          

<ipython-input-5-55987713a981>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices_claims_last_actived['invoiced_amount'] = invoices_claims_last_actived['invoiced_unit_price'] * invoices_claims_last_actived['invoiced_units']
<ipython-input-5-55987713a981>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices_claims_last_actived['claim_start_date'] = invoices_claims_last_actived['claim_start_date'].astype('datetime64[ns]').dt.date
<ipython-input-5-55987713a981>:58: SettingWithCopyWarning: 
A valu

In [6]:
# Select just the fields to be used
invoices_claims_last_completed = invoices_claims_last_completed_all_fields[{"member_id"
                                                                          , "invoice_state"
                                                                          , "claim_state"
                                                                          , "key"  
                                                                          , "key.1"
                                                                          , "key.2"
                                                                          , "claim_start_date"
                                                                          , "claim_end_date"
                                                                          , "claim_funded_amount"  
                                                                          , "funded_total"
                                                                          , "invoiced_units"
                                                                          , "invoiced_unit_price"
                                                                          }]

# Rename some dataSet colums to create a name's pattern
invoices_claims_last_completed.rename(columns={'funded_total':'invoice_amount'
                                           , 'claim_funded_amount':'funded_amount'
                                           , 'key.1':'claim_level2_key'
                                           , 'key.2':'claim_level1_key'
                                           , 'key':'claim_level3_key'
                                              } , inplace = True)

invoices_claims_last_completed['invoiced_amount'] = invoices_claims_last_completed['invoiced_unit_price'] * invoices_claims_last_completed['invoiced_units'] 

# Get the name of columns with Nan values
#  and figured out that some fields are Nan values
df_any_Nan = invoices_claims_last_completed.isna().any()
df_any_Nan


# Check The NaN fields
df_invoices_claims_filtered = invoices_claims_last_completed[invoices_claims_last_completed.isna().any(axis=1)]

# Check state of claims with NaN claim_state 
df_filtered = df_invoices_claims_filtered[{"invoice_state"
                                         , "claim_state"
                                         }]

# Group the state of invoice and claim of all null data
#  And Keeps these records
#   Because if 'funded_amount' is NaN means the claims is not PAID OR REFUNDED
df_filtered_grouped = df_invoices_claims_filtered.groupby(["invoice_state"
                                                         , "claim_state"
                                                          ]).count()

# Some funded_amount == null, then they will be assinged as 0 and will not be deleted
# Because if they are null means they were not paid
invoices_claims_last_completed.loc[invoices_claims_last_completed['funded_amount'].isnull(),['funded_amount']] = 0

#invoices_claims_last_completed['invoice_created_at'] = invoices_claims_last_completed['invoice_created_at'].astype('datetime64[ns]').dt.date

invoices_claims_last_completed['claim_start_date'] = invoices_claims_last_completed['claim_start_date'].astype('datetime64[ns]').dt.date
invoices_claims_last_completed['claim_end_date'] = invoices_claims_last_completed['claim_end_date'].astype('datetime64[ns]').dt.date

# Group the invoices from their claims
invoices_claims_last_completed_cleased = invoices_claims_last_completed.groupby(["member_id"
                                                                               , "invoice_state" 
                                                                               , "claim_state"
                                                                               , "claim_level1_key"
                                                                               , "claim_level2_key"
                                                                               , "claim_level3_key"]).agg({"invoiced_amount": "sum", "funded_amount": "sum"}).reset_index()

<ipython-input-6-48f7ed1e7734>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices_claims_last_completed['invoiced_amount'] = invoices_claims_last_completed['invoiced_unit_price'] * invoices_claims_last_completed['invoiced_units']
<ipython-input-6-48f7ed1e7734>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invoices_claims_last_completed['claim_start_date'] = invoices_claims_last_completed['claim_start_date'].astype('datetime64[ns]').dt.date
<ipython-input-6-48f7ed1e7734>:54: SettingWithCopyWarnin

### Merge dataSets cleased

In [7]:
# Member dataSet with plans and budgets dataSet
df_members_plans_budgets_merged = pd.merge(df_members_cleased, df_plans_budgets_cleased, on=["member_key", "member_key"])            

In [8]:
invoices_claims_plan_delivery_active = df_members_plans_budgets_merged.loc[(df_members_plans_budgets_merged['plan_status'] == 'PLAN_DELIVERY_ACTIVE')]

invoices_claims_plan_delivery_active = invoices_claims_plan_delivery_active.merge(invoices_claims_last_actived_cleased, on='member_id')

invoices_claims_plan_delivery_active = invoices_claims_plan_delivery_active[
            (invoices_claims_plan_delivery_active.budget_level1_key == invoices_claims_plan_delivery_active.claim_level1_key)

         &  (invoices_claims_plan_delivery_active.budget_level2_key == invoices_claims_plan_delivery_active.claim_level2_key)

         &  ((invoices_claims_plan_delivery_active.budget_level3_key == 'No Assigned')
          | ((invoices_claims_plan_delivery_active.budget_level3_key != 'No Assigned')
           & (invoices_claims_plan_delivery_active.budget_level3_key == invoices_claims_plan_delivery_active.claim_level3_key)))
        
           ]

In [9]:
invoices_claims_plan_completed = df_members_plans_budgets_merged.loc[(df_members_plans_budgets_merged['plan_status'] == 'COMPLETED')]

invoices_claims_plan_completed = invoices_claims_plan_completed.merge(invoices_claims_last_completed_cleased, on='member_id')

invoices_claims_plan_completed = invoices_claims_plan_completed[
            (invoices_claims_plan_completed.budget_level1_key == invoices_claims_plan_completed.claim_level1_key)

         &  (invoices_claims_plan_completed.budget_level2_key == invoices_claims_plan_completed.claim_level2_key)

         &  ((invoices_claims_plan_completed.budget_level3_key == 'No Assigned')
          | ((invoices_claims_plan_completed.budget_level3_key != 'No Assigned')
           & (invoices_claims_plan_completed.budget_level3_key == invoices_claims_plan_completed.claim_level3_key)))
        
           ]

In [10]:
frames = [invoices_claims_plan_delivery_active, invoices_claims_plan_completed]
invoices_claims_merged = pd.concat(frames)

In [13]:
df_model = invoices_claims_merged.groupby(["member_key"
                                         , "first_name"
                                         , "last_name"
                                         , "disabilities"
                                         , "SA1"
                                         , "price_zone_code"                                         
                                         , "plan_key"
                                         , "plan_status"
                                         , "plan_start_date"
                                         , "plan_end_date"                                                                               
                                         , "budget_level3_name"
                                         , "budget_level3_key"
                                         , "budget_level2_key"
                                         , "budget_level2_name"
                                         , "budget_level1_key"
                                         , "budget_level1_name"
                                         , "budget_value_allocated"  	
                                         , "invoice_state"
                                         , "claim_state"
                                          ]).agg({"invoiced_amount": "sum", "funded_amount": "sum"}).reset_index()                                                                 

In [14]:
##**** TESTE

df_model.loc[(df_model['member_key'] == '007c1090-d6ca-11ea-bb8a-a73bf14c75f9')
                                              & (df_model['budget_level2_key'] == '7d0af57b-f597-11e9-bfb4-022d4762bb3c')         
                                 ]  

,member_key,first_name,last_name,disabilities,SA1,price_zone_code,plan_key,plan_status,plan_start_date,plan_end_date,...,budget_level3_key,budget_level2_key,budget_level2_name,budget_level1_key,budget_level1_name,budget_value_allocated,invoice_state,claim_state,invoiced_amount,funded_amount
14,007c1090-d6ca-11ea-bb8a-a73bf14c75f9,Janean,Potzold,No Assigned,No Assigned,ACT_NSW_QLD_VIC,812de63d-76f2-4818-96f8-c92ab85c16a0,PLAN_DELIVERY_ACTIVE,2020-08-18,2021-08-18,...,No Assigned,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,24504.0,ALL_PAID,PAID,6886.650,6886.68
15,007c1090-d6ca-11ea-bb8a-a73bf14c75f9,Janean,Potzold,No Assigned,No Assigned,ACT_NSW_QLD_VIC,812de63d-76f2-4818-96f8-c92ab85c16a0,PLAN_DELIVERY_ACTIVE,2020-08-18,2021-08-18,...,No Assigned,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,24504.0,INVOICE_REJECTED,SCHEME_REJECTED,387.980,0.00
16,007c1090-d6ca-11ea-bb8a-a73bf14c75f9,Janean,Potzold,No Assigned,No Assigned,ACT_NSW_QLD_VIC,812de63d-76f2-4818-96f8-c92ab85c16a0,PLAN_DELIVERY_ACTIVE,2020-08-18,2021-08-18,...,No Assigned,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,24504.0,PENDING_CLAIM,OPERATOR_AUTHORIZED,193.990,0.00
19,007c1090-d6ca-11ea-bb8a-a73bf14c75f9,Janean,Potzold,No Assigned,No Assigned,ACT_NSW_QLD_VIC,8910b6c2-643b-421e-9da6-8f2fea8f3e9a,COMPLETED,2020-08-14,2020-08-17,...,No Assigned,7d0af57b-f597-11e9-bfb4-022d4762bb3c,Improved daily living skills,5874551b-f597-11e9-bfb4-022d4762bb3c,Capacity Building,8168.0,ALL_PAID,PAID,678.965,678.97
